In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.environ['USE_PYGEOS'] = '0'
import pysal.lib
import helpers as hs
from importlib import reload
import folium
import pandas as pd
import geopandas as gpd

In [ ]:
%cd ../data

Read geopackage for PC witout paris

In [ ]:
pc_sans_paris_gdf = gpd.read_file("pc_sans_paris_gdf.gpkg", layer="cool")

In [ ]:
import r5py
import shapely

In [ ]:
transport_network = r5py.TransportNetwork("Petite-Couronne-Sans-Paris.pbf", ["IDFM.gtfs.zip"])

In [ ]:
hs.folium_grid_cat_plot(pc_sans_paris_gdf,"restaurant")

In order to work with r5 we replace IdINSIPRE with id

In [ ]:
pc_sans_paris_gdf.rename(columns={'IdINSPIRE':'id'}, inplace = True)

In [ ]:
pc_sans_paris_gdf['Log'] = pc_sans_paris_gdf['Log_inc'] + pc_sans_paris_gdf['Log_av45'] + pc_sans_paris_gdf['Log_45_70'] + pc_sans_paris_gdf['Log_70_90'] + pc_sans_paris_gdf['Log_ap90']


In [ ]:
import datetime
import time

In [ ]:
pc_sans_paris_gdf_centered = pc_sans_paris_gdf.copy()
pc_sans_paris_gdf_centered['geometry'] = pc_sans_paris_gdf_centered['geometry'].apply(lambda x : x.centroid)

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=pc_sans_paris_gdf_centered,
    destinations=pc_sans_paris_gdf_centered,
    departure=datetime.datetime(2024,10,8,11,59),
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
)

Compute travel times:

In [ ]:
travel_times = travel_time_matrix_computer.compute_travel_times()

In [ ]:
travel_times.to_csv("travel_times_pc_sp.csv")

In [ ]:
travel_times_square = hs.vert_to_square(travel_times)

In [ ]:
weights_by_id = hs.transfer_time_to_weight_faster(travel_times_square)

In [ ]:
df_reset = weights_by_id.reset_index()

In [ ]:
pc_sans_paris_gdf.

In [ ]:
interestVar = ['Log','Log_soc','restaurant',
       'culture and art', 'education', 'food_shops', 'fashion_beauty',
       'supply_shops']
accessibility_measures = hs.calculate_2SFCA_accessibility(pc_sans_paris_gdf,interestVar,weights_by_id)

In [ ]:
for var in interestVar:
    pc_sans_paris_gdf[var+"_access"] = accessibility_measures[var]

In [ ]:
reload(hs)

In [ ]:
hs.folium_color_map(pc_sans_paris_gdf,"Log_soc_access")

In [ ]:
interestVar2 = ['restaurant',
       'culture and art', 'education', 'food_shops', 'fashion_beauty',
       'supply_shops']
for var in interestVar2:
        pc_sans_paris_gdf[str("weight_" + var)] = pc_sans_paris_gdf[var].sum()/(pc_sans_paris_gdf[interestVar2].sum(axis = 1).sum())

In [ ]:
1 - pc_sans_paris_gdf[[str("weight_" + var) for var in interestVar2]]

In [ ]:
for var in interestVar2 : 
    pc_sans_paris_gdf[str(var + "_access" + "_norm*weight")] = (pc_sans_paris_gdf[str(var + "_access")].apply(lambda x : (x - pc_sans_paris_gdf[str(var + "_access")].min())/(pc_sans_paris_gdf[str(var + "_access")].max() - pc_sans_paris_gdf[str(var + "_access")].min())))*(1 - pc_sans_paris_gdf[str("weight_" + var)])

In [ ]:
for var in interestVar2 : 
    pc_sans_paris_gdf[str(var + "_access" + "without_norm*weight")] = (pc_sans_paris_gdf[str(var + "_access")].apply(lambda x : (x - pc_sans_paris_gdf[str(var + "_access")].min())/(pc_sans_paris_gdf[str(var + "_access")].max() - 
                                                                                                                                       pc_sans_paris_gdf[str(var + "_access")].min())))

In [ ]:
pc_sans_paris_gdf["CS_aggregated"] = pc_sans_paris_gdf[[i + "_access_norm*weight" for i in interestVar2]].sum(axis = 1) 

In [ ]:
pc_sans_paris_gdf["CS_aggregated_without_weight"] = pc_sans_paris_gdf[[i + "_accesswithout_norm*weight" for i in interestVar2]].sum(axis = 1) 

In [ ]:
hs.folium_color_map(pc_sans_paris_gdf,"CS_aggregated_without_weight")

In [ ]:
hs.folium_color_map(pc_sans_paris_gdf,"CS_aggregated")